In [8]:
from tetra.model import val
from tetra.model import inference
from tetra.data import data_loader
from tetra.utils import file_io
from pathlib import Path
import importlib
import shutil
import os
from pycocotools.coco import COCO

In [56]:
importlib.reload(inference)

<module 'tetra.model.inference' from '/Users/mateobigearddasen/Documents/agrosup_stage/Tetra_Project/src/tetra/model/inference.py'>

In [6]:
model = os.path.abspath("../../runs/train/lyrurus_yolov11l_background_dataset/weights/best.pt")
dataset = Path("../../dataset/og/images")
label = Path("../../dataset/inference/labels/lyrurus_yolov11l_background_labels")
gt_annotations = "../../dataset/og/_annotations.coco.json"

In [58]:
#Select images of the dataset
imgs = data_loader.random_data_sample(dataset, 0.025, [".jpg"])

In [59]:
#Change images directory
for img in img_samples:
    img = Path(img)
    image = dataset / img
    shutil.move(image, Path(inference_dataset) / img.name)

In [10]:
#Get ground truth annotations
coco_gt = COCO(gt_annotations)
#Map id to image name
file_to_id = {img["file_name"]: img["id"] for img in coco_gt.dataset["images"] if img["file_name"]}

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [ ]:
#Inference of the data 
predictions = []
i = 0
for f in Path(dataset).iterdir():
    if f.suffix.lower() == ".jpg":
        prediction = inference.solo_inference(model, str(f))
        prediction = prediction.to_coco_annotations()
        #Save the prediction in yolo
        yolo_pred = file_io.coco_to_yolo(prediction, f, True)
        file_io.save_yolo_labels(yolo_pred, label, f.stem + ".txt")
        for ann in prediction:
            #Convert id to the gt id
            ann["image_id"] = file_to_id[f.name]
            ann["category_id"] = 1
            #Change category name
            ann["category_name"] = "Lyrurus_tetrix"
        predictions.append(prediction)
        i += 1
        if i % 100 == 0:
            print(f"IMAGES {i} ")
print("END")

#Save predictions
file_io.save_coco_labels(file_io.flatten_pred(predictions), "./annotations", "lyrurus_yolov11l_background_dataset.coco.json" )

IMAGES 100 
IMAGES 200 
IMAGES 300 
IMAGES 400 
IMAGES 500 


In [62]:
#Save predictions
file_io.save_coco_labels(file_io.flatten_pred(predictions), "./annotations", "lyrurus_yolov11l_background_dataset.coco.json" )

In [21]:
#Filter predictions
filtered_predictions = [ann for ann in file_io.flatten_pred(predictions) if ann["score"] > 0.7]

In [9]:
metric_val = val.eval_hr("../../dataset/og/_annotations.coco.json", "../../dataset/inference/labels/lyrurus_yolov11l_background/lyrurus_yolov11l_background_dataset2.coco.json")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!


In [10]:
metric_val.evaluate()
metric_val.accumulate()
metric_val.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.327
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.213
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.214
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.205
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.084
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.227
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.227
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.191
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100